In [1]:
#Imports & Dependencies
!pip install selenium
!pip install splinter
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430
[WDM] - Driver [C:\Users\Viji\.wdm\drivers\chromedriver\win32\90.0.4430.24\chromedriver.exe] found in cache


In [3]:
browser = Browser("chrome", **executable_path, headless=False)

# NASA Mars News

In [4]:
news_url = "https://redplanetscience.com/"
browser.visit(news_url)
html = browser.html
soup = BeautifulSoup(html, "html.parser")



In [5]:
article = soup.find("div", class_='list_text')
news_title = article.find("div", class_="content_title").text
news_p = article.find("div", class_ ="article_teaser_body").text



In [6]:
print(news_title)

Mars Helicopter Attached to NASA's Perseverance Rover


In [7]:
print(news_p)

The team also fueled the rover's sky crane to get ready for this summer's history-making launch.


# JPL Mars Space Images - Featured Image

In [8]:
image_url = "https://spaceimages-mars.com/"
browser.visit(image_url)
html = browser.html
soup = BeautifulSoup(html, "html.parser")

In [9]:
image = soup.find("img", class_="headerimage fade-in")["src"]
featured_image_url = "https://spaceimages-mars.com/" + image
print(featured_image_url)



https://spaceimages-mars.com/image/featured/mars2.jpg


# Mars Facts

In [10]:
facts_url = 'https://galaxyfacts-mars.com/'
# Visit the Mars Facts Site Using Pandas to Read
mars_facts = pd.read_html("https://galaxyfacts-mars.com/")[0]
mars_facts.reset_index(inplace=True)
mars_facts.columns=["ID", "Properties", "Mars", "Earth"]
mars_facts

,ID,Properties,Mars,Earth
0,0,Mars - Earth Comparison,Mars,Earth
1,1,Diameter:,"6,779 km","12,742 km"
2,2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,3,Moons:,2,1
4,4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,5,Length of Year:,687 Earth days,365.24 days
6,6,Temperature:,-87 to -5 °C,-88 to 58°C


# Mars Hemispheres

In [11]:
hemispheres_url = 'https://marshemispheres.com/'
browser.visit(hemispheres_url)
hemisphere_html = browser.html
hemisphere_soup = BeautifulSoup(hemisphere_html,'html.parser')
print(hemisphere_soup)

<html lang="en"><head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<link href="css/jquery-ui.css" rel="stylesheet" type="text/css"/>
<title>Astropedia Search Results | GUSS Astrogeology Science Center</title>
<meta content="GUSS Astrogeology Science Center Astropedia search results." name="description"/>
<meta content="GUSS,Astrogeology Science Center,Cartography,Geology,Space,Geological Survey,Mapping" name="keywords"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
<link href="css/main.css" media="screen" rel="stylesheet"/>
<link href="css/print.css" media="print" rel="stylesheet"/>
<link href="#" rel="icon" type="image/x-ico"/>
</head>
<body id="results">
<header>
<a href="#" style="float:right;margin-top:10px;" target="_blank">
<img alt="USGS: Science for a Changing World" class="logo" height="60" src="images/usgs_logo_main_2x.png"/>
</a>
<a href="#" style="float:

In [12]:
hemisphere_names = []

# Search for the names of all four hemispheres
results = hemisphere_soup.find_all('div', class_="collapsible results")
hemispheres = results[0].find_all('h3')

# Get text and store in list
for name in hemispheres:
    hemisphere_names.append(name.text)

hemisphere_names

['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

In [13]:
# Search for thumbnail links
thumbnail_results = results[0].find_all('a')
thumbnail_links = []

for thumbnail in thumbnail_results:
    
    # If the thumbnail element has an image...
    if (thumbnail.img):
        
        # then grab the attached link
        thumbnail_url = 'https://marshemispheres.com/' + thumbnail['href']
        
        # Append list with links
        thumbnail_links.append(thumbnail_url)

thumbnail_links

['https://marshemispheres.com/cerberus.html',
 'https://marshemispheres.com/schiaparelli.html',
 'https://marshemispheres.com/syrtis.html',
 'https://marshemispheres.com/valles.html']

In [14]:
full_imgs = []

for url in thumbnail_links:
    
    # Click through each thumbanil link
    browser.visit(url)
    
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    # Scrape each page for the relative image path
    results = soup.find_all('img', class_='wide-image')
    relative_img_path = results[0]['src']
    
    # Combine the reltaive image path to get the full url
    img_link = 'https://astrogeology.usgs.gov/' + relative_img_path
    
    # Add full image links to a list
    full_imgs.append(img_link)

full_imgs

['https://astrogeology.usgs.gov/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg',
 'https://astrogeology.usgs.gov/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg',
 'https://astrogeology.usgs.gov/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg',
 'https://astrogeology.usgs.gov/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg']

In [15]:
# Zip together the list of hemisphere names and hemisphere image links
mars_hemi_zip = zip(hemisphere_names, full_imgs)

hemisphere_image_urls = []

# Iterate through the zipped object
for title, img in mars_hemi_zip:
    
    hemisphere_dict = {}
    
    # Add hemisphere title to dictionary
    hemisphere_dict['title'] = title
    
    # Add image url to dictionary
    hemisphere_dict['img_url'] = img
    
    # Append the list with dictionaries
    hemisphere_image_urls.append(hemisphere_dict)

hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]